# Tabu Search

## Code

In [ ]:
import math, time, random, sys
from random import randint, shuffle
import networkx as graphs
import pandas as pd
import numpy as np

class CompleteWGraph:
    n = 0
    p = 0
    lower_weight = 0
    upper_weight = 0
    distmatrix = {}
    w_edges = []
    def __init__(self,n,p,lower_weight,upper_weight):
        """n: number of nodes
        p: prob of 2 nodes being connected (between 0-1)
        lower/upper weight: range of possible weight values"""
        self.n = n
        self.p = p
        self.lower_weight= lower_weight
        self.upper_weight = upper_weight
    def random_weighted_graph(self):
        g = graphs.gnp_random_graph(self.n,self.p)
        m = g.number_of_edges()
        weights = [random.randint(self.lower_weight, self.upper_weight) for r in range(m)]
        #unweighted connections
        uw_edges = g.edges()
        # Create weighted graph edge list
        i=0
        w_edges = []
        ret_graph = graphs.Graph()
        for edge in uw_edges:
        #w_edges = [uw_edges[i][0], uw_edges[i][1], weights[i]]
        #w_edges+={(edge[0],edge[1]):weights[i]}
            ret_graph.add_edge(edge[0],edge[1],weight=weights[i])
            i =i +1
        #print(w_edges)
        #return graphs.Graph(w_edges, weighted = True,s=weights)
        return ret_graph

In [ ]:
#g1_data = CompleteWGraph(10,0.6,5,20)
#g1 = g1_data.random_weighted_graph()

#weightdict = g1.get_edge_data(0,2)
#print(weightdict)
#print(g1.get_edge_data(0,2)['weight'])

#g1_data.createDistanceMatrix(g1)
#print("g1 distance matrix")
#print(g1_data.distmatrix)

###  Data Format is dict:
#           data[node_name] = gives you a list of link info
#           data[link_index][0] = name of node that edge goes to
#           data[link_index][1] = weight of that edge
def cook_data(nodes, probability, min_weight, max_weight):
    linkset = []
    links = {}

    if min_weight>max_weight:
        print('Lower weight cannot be greater then upper weight for the weight range. ')
        sys.exit()
    if probability<0 or probability>1:
        print('Probability incorrect. Must be between 0 and 1. ')
        sys.exit()
    g1_data = CompleteWGraph(nodes, probability, min_weight, max_weight)
    g1 = g1_data.random_weighted_graph()
    node_list=list(g1.nodes())
    max_weight_of_all=0
    for a in node_list:
        for b in node_list:
            if a==b:
                continue
            link = []
            link.append(a)
            link.append(b)
            edge_weight=g1.get_edge_data(a,b)['weight']
            link.append(edge_weight)
            linkset.append(link)
            print('%d %d %d' % (a,b,edge_weight))
            if edge_weight>max_weight_of_all:
                max_weight_of_all=edge_weight

    for link in linkset:
        try:
            linklist = links[str(link[0])]
            linklist.append(link[1:])
            links[str(link[0])] = linklist
        except:
            links[str(link[0])] = [link[1:]]

    return links, max_weight_of_all

In [3]:
def tabu_search(nodes, probability, min_weight, max_weight):
    global max_fitness, start_node
    graph, max_weight = cook_data(nodes, probability, min_weight, max_weight)

    ## Below, get the keys (node names) and shuffle them, and make start_node as start
    s0 = list(graph.keys())
    shuffle(s0)

    if int(s0[0]) != start_node:
        for i in range(len(s0)):
            if  int(s0[i]) == start_node:
                swap = s0[0]
                s0[0] = s0[i]
                s0[i] = swap
                break;

    # max_fitness will act like infinite fitness
    max_fitness = ((max_weight) * (len(s0)))+1
    sBest = s0
    vBest = fitness(s0, graph)
    bestCandidate = s0
    tabuList = []
    tabuList.append(s0)
    stop = False
    best_keep_turn = 0

    start_time = time.time()
    while not stop :
        sNeighborhood = getNeighbors(bestCandidate)
        bestCandidate = sNeighborhood[0]
        for sCandidate in sNeighborhood:
            if (sCandidate not in tabuList) and ((fitness(sCandidate, graph) < fitness(bestCandidate, graph))):
                bestCandidate = sCandidate

        if (fitness(bestCandidate, graph) < fitness(sBest, graph)):
            sBest = bestCandidate
            vBest = fitness(sBest, graph)
            best_keep_turn = 0

        tabuList.append(bestCandidate)
        if (len(tabuList) > maxTabuSize):
            tabuList.pop(0)

        if best_keep_turn == stoppingTurn:
            stop = True

        best_keep_turn += 1

    exec_time =  time.time() - start_time
    return sBest, vBest, exec_time



## Tabu Search Takes edge-list in a given format:
#nodefrom nodeto weight
#0 1 5
#3 2 4
#1 0 3
#Undirectional edges should be written 2 times for both nodes.
maxTabuSize = 10000
neighborhood_size = 500
stoppingTurn = 500
max_fitness = 0
start_node = 0
solution, value, exec_time = tabu_search(nodes=5, probability=1, min_weight=10, max_weight=20)

print(solution)
print('----> '.join(a for a in solution))
print('Shortest Distance : %d' %(value))
print('Solved in %s seconds'%exec_time)

0 1 16
0 2 20
0 3 13
0 4 13
1 0 16
1 2 11
1 3 16
1 4 18
2 0 20
2 1 11
2 3 20
2 4 20
3 0 13
3 1 16
3 2 20
3 4 15
4 0 13
4 1 18
4 2 20
4 3 15
['0', '4', '3', '1', '2']
0----> 4----> 3----> 1----> 2
Shortest Distance : 75
Solved in 0.8946053981781006 seconds


## References

* Ozan Polatbilek, (Jul 9, 2019) *Tabu search on Travelling Salesman Problem* [online] Available at: <https://github.com/polatbilek/Tabu-search-on-Travelling-Salesman-Problem/blob/master/tabu_search.py> [Accessed on Apr 8, 2020]